In [ ]:
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import math

In [ ]:
_, blue = cv2.imreadmulti('data/Blue-34a-dbl_0002.tiff', [], cv2.IMREAD_UNCHANGED)
_, red = cv2.imreadmulti('data/Red-34a-dbl_0002.tiff', [], cv2.IMREAD_UNCHANGED)
_, composite = cv2.imreadmulti('data/Composite-34a-dbl_0002.tiff', [], cv2.IMREAD_GRAYSCALE)
# List of tuples of the form (blue, red, composite) 
blue_composite = [(blue[i], composite[i]) for i in range(len(composite))]
red_composite = [(red[i], composite[i]) for i in range(len(composite))]
red_blue_composite = [(red[i], blue[i], composite[i]) for i in range(len(composite))]

In [ ]:
#Blue
points_blue = []
for idx, (bl, cmp) in enumerate(blue_composite):
    if idx < len(blue_composite)*0.05 or idx > len(blue_composite)*0.95:
        point = get_intensity_points(bl, cmp, outer_bool=True)
    else:
        point = get_intensity_points(bl, cmp)
    points_blue.append(point)

#Red
points_red = []
for idx, (rd, cmp) in enumerate(red_composite):
    if idx < len(red_composite)*0.05 or idx > len(red_composite)*0.95:
        point = get_intensity_points(rd, cmp, outer_bool=True)
    else:
        point = get_intensity_points(rd, cmp)
    points_red.append(point)

In [ ]:
def get_intensity_pictures(img,composite,outer_bool=False):  
    _ , thresh = cv2.threshold(composite, 5, 10, cv2.THRESH_BINARY)
    contours,_ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    #Find biggest contour
    contours = [max(contours, key = cv2.contourArea)]

    black = np.full(thresh.shape, 0, dtype=np.uint8)
    if outer_bool:
        return (black , composite, composite)

    white_outer_blackbg = cv2.drawContours(black, contours, -1, 255, -1)
    clean_composite = cv2.bitwise_and(composite, white_outer_blackbg)

    left = (math.inf,0)
    right = (0,0)
    top = (0, math.inf)
    bottom = (0,0)
    threshold_intensity = 10
    #Find left most edge
    for y, row in enumerate(clean_composite):
        for x, pixel in enumerate(row):
            if pixel >= threshold_intensity and x < left[0]:
                left = (x,y)
            #Find right most edge
            if pixel >= threshold_intensity and x > right[0]:
                right = (x,y)
            #Find top most edge
            if pixel >= threshold_intensity and y < top[1]:
                top = (x,y)
            #Find bottom most edge
            if pixel >= threshold_intensity and y > bottom[1]:
                bottom = (x,y)

    #Given four points find top left and bottom right of the rectangle
    def find_top_left_bottom_right(p1,p2,p3,p4):
        top_left = (min(p1[0],p2[0],p3[0],p4[0]),min(p1[1],p2[1],p3[1],p4[1]))
        bottom_right = (max(p1[0],p2[0],p3[0],p4[0]),max(p1[1],p2[1],p3[1],p4[1]))
        return top_left, bottom_right

    def x_radius(p1,p2):
        return (p2[0]-p1[0])/2
    def y_radius(p1,p2):
        return (p2[1]-p1[1])/2

    top_left, bottom_right = find_top_left_bottom_right(left,right,top,bottom)

    left_2 = (left[0] + x_radius(top_left,bottom_right)/2, left[1])
    right_2 = (right[0] - x_radius(top_left,bottom_right)/2, right[1])
    top_2 = (top[0], top[1] + y_radius(top_left,bottom_right)/2)
    bottom_2 = (bottom[0], bottom[1] - y_radius(top_left,bottom_right)/2)
    top_left_2, bottom_right_2 = find_top_left_bottom_right(left_2,right_2,top_2,bottom_2) 
    center = (int((top_left_2[0] + bottom_right_2[0])/2), int((top_left_2[1] + bottom_right_2[1])/2))

    #Make a mask of the ellipse that sets everything outside the ellipse to 0
    mask = np.zeros(composite.shape, dtype=np.uint8)
    mask = cv2.ellipse(mask,
                         center,
                         (int(x_radius(top_left_2,bottom_right_2)), int(y_radius(top_left_2,bottom_right_2)))
                         , 0, 0, 360, (255,255 ,255), -1)

    #Apply the mask to the image
    masked_image = cv2.bitwise_and(composite, mask)

    #Find biggest contour
    contours = [max(contours, key = cv2.contourArea)]

    # Draw contours + hull results
    mask_2 = np.zeros(thresh.shape, dtype=np.uint8)

    cv2.drawContours(mask_2, contours, -1, (255,255 ,255), -1)
    masked_image_2 = cv2.bitwise_and(composite, mask_2)

    #Remove inner from outer
    mask_1_1 = cv2.bitwise_not(mask)
    outer_only = cv2.bitwise_and(masked_image_2, mask_1_1)

    return (masked_image, outer_only, composite)